In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ASUS\\OneDrive\\Desktop\\Text-Summariser-newton main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ASUS\\OneDrive\\Desktop\\Text-Summariser-newton main'

In [5]:
from src.textsummarisernewton.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textsummarisernewton.utils.common import read_yaml, create_directories
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

class ConfigurationManager:
    def __init__(self, config_filepath: Path = CONFIG_FILE_PATH, params_filepath: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.textsummarisernewton.logging.logger import logger
from src.textsummarisernewton.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        
   
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


2024-07-29 01:14:45,275 - textsummarisernewton.logging.logger - INFO - YAML file: config\config.yaml loaded successfully


[2024-07-29 01:14:45,275: INFO: common: YAML file: config\config.yaml loaded successfully]


2024-07-29 01:14:45,281 - textsummarisernewton.logging.logger - INFO - YAML file: params.yaml loaded successfully


[2024-07-29 01:14:45,281: INFO: common: YAML file: params.yaml loaded successfully]


2024-07-29 01:14:45,287 - textsummarisernewton.logging.logger - INFO - created directory at: artifacts


[2024-07-29 01:14:45,287: INFO: common: created directory at: artifacts]


2024-07-29 01:14:45,291 - textsummarisernewton.logging.logger - INFO - created directory at: artifacts/data_ingestion


[2024-07-29 01:14:45,291: INFO: common: created directory at: artifacts/data_ingestion]


2024-07-29 01:14:45,296 - src.textsummarisernewton.logging.logger - INFO - File already exists of size: ~ 7718.35 KB


[2024-07-29 01:14:45,296: INFO: 1447554207: File already exists of size: ~ 7718.35 KB]
